# Advanced Macro I


In [ ]:
# Import necessary libraries
# Plotting library, including specific imports for animations
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.animation as animation
# Numerical computing library
import numpy as np
# Special option to display animations within a Colab notebok
rc('animation', html='jshtml')

In [ ]:
# Set the exogenous variables
theta = 0.5   # 1/theta is willingness to shift consumption between different periods
rho = 0.01    # Discount rate
n = 0.009     # Household growth rate
g = 0.002     # Technology growth rate
alpha = 0.2   # Capital share of output

# Make sure the parameters are within the allowed ranges
if not (0 < alpha < 1):
  raise "alpha must be between 0 and 1"
if not theta > 0:
  raise "theta must be positive"
if not rho - n - (1 - theta) * g > 0:
  raise "rho - n - (1 - theta) * g must be positive"

# Laws of motion of the economy
Following Romer (2012, chapter 2.2), recall that the equations governing the dynamics of the economy are given by equation (2.24):
$$
\frac{\dot{c}(t)}{c(t)} = \frac{f^\prime(k(t)) - \rho - \theta g}{\theta}
$$

and equation (2.25):
$$\dot{k}(t) = f(k(t)) - c(t) - (n + g) k(t)$$

These two equations build a *system of differential equations*. Solving such systems by hand is generally hard, however, with programming languages we can let computers do the heavy lifting for us.

In this notebook, we will take a look at a graphical solution for these equations, also known phase diagrams, as well as a numerical solution using Python. While there exist packages to solve differential equations such as [SciPy](https://www.scipy.org/), we will write our own implementation as this is more educational.


### Finding the level of $k$ that implies $\dot{c} = 0$
Consider the equation (2.24) in Romer:
$$
\frac{\dot{c}(t)}{c(t)} = \frac{f^\prime(k(t)) - \rho - \theta g}{\theta}
$$
Given this equation, we can find $k^\star$, the level of $k$ for which $\dot{c} = 0$, this is given by:
$$\begin{aligned}
0 &= \frac{f^\prime(k^\star) - \rho - \theta g}{\theta} \\
k^\star &= f^{\prime{-1}}(\rho + \theta g)
\end{aligned}
$$
If we take $f(x) = x^\alpha$ to be the intensive form of the production function, this gives the first derivative
$f^\prime(x) = \alpha x^{\alpha-1}$.  

Hence $f^{\prime{-1}}(x) = \left(\frac{x}{\alpha}\right)^\frac{1}{\alpha-1}$ and $k^\star = f^{\prime{-1}}(\rho + \theta g) = \left(\frac{\rho+\theta g}{\alpha}\right)^\frac{1}{\alpha-1}$.

In [ ]:
# Create a function to compute f' inverse
def f_prime_inv(x):
  return (x / alpha)**(1 / (alpha - 1))
# Compute k* using the above function
k_star = f_prime_inv(rho + theta * g)
k_star  # Display the result

37.54452707246351

## Finding the level of $c$ that implies $\dot{k} = 0$

Similarly, we can find the level of $c$ that implies $\dot{k} = 0$ by considering the equation (2.5) in Romer:
$$\dot{k}(t) = f(k(t)) - c(t) - (n + g) k(t)$$

We denote the level of $c$ which gives $\dot{k} = 0$ by $c^\star$ and it is equal to:
$$\begin{aligned}
0 &= f(k(t)) - c^\star - (n + g) k(t) \\
c^\star &= f(k(t)) - (n + g) k(t)
\end{aligned}$$

While we could represent $k^\star$ by a constant, $c^\star$ depends on the level of $k(t)$, hence we can only represent it as a function of $k$.

In [ ]:
# Create a function to compute f, the intensive form of the production function
def f(x):
  return x**alpha
# Create a function to compute c* given the level of k
def c_star(k):
  return f(k) - (n + g) * k

## Plotting the dynamics of $c$ and $k$
Now that we have computed the necessary information, we can replicate the figure 2.3 in Romer given our levels of $\theta$, $\rho$, $n$, $g$, and $\alpha$ (which can be changed freely in the first cell of this notebook, just make sure to re-run all cells after making changes).

Note that $c^\star = 0$ happens either at $k(t) = 0$ or $k(t) = (n + g)^{\frac{1}{\alpha-1}}$, hence we must define the x-axis on our plot to range from $0$ to $(n + g)^{\frac{1}{\alpha-1}}$ if we want to show the whole curve where $\dot{k} = 0$ for any parameter combination.

In [ ]:
# 1000 linearly spaced numbers between both points where c* = 0, this will be our x-axis
k = np.linspace(0, pow(n + g, 1 / (alpha - 1)), 1000)
# Set up a figure
fig, ax = plt.subplots(figsize = (12, 8))
# Add a green line with x values given by k and y values given by c*(k)
ax.plot(k, c_star(k), color = 'green', label = "k̇ = 0")
# Add a vertical line at k*
ax.axvline(k_star, color = 'blue', label = "ċ = 0")
ax.set_xlabel("k")                        # Add x-axis label
ax.set_ylabel("c")                        # Add y-axis label
ax.set_title("The dynamics of c and k")   # Add plot title
ax.grid()                                 # Add a grid in the background
ax.legend()                               # Add legend (show labels top right)
plt.show()                                # Display the plot

The plot is similar to that of figure 2.3 in Romer, however, we have not yet added the arrows which show the dynamics of the system. While we could add them manually, this is bothersome as we would need to write some code for each individual arrow. Instead, we'll make use of the tools `matplotlib` offers and create a so called *quiver plot*. Quiver plots are vector fields which can be used to show the dynamics of a system and are very common when dealing with differential equations.

Quiver plots allow us to plot a grid of arrows/vectors, where each vector represents both the direction (vector orientation) and intensity (vector length) of changes within the dynamical system.

In [ ]:
# In order to compute the direction and intensity of changes within the system,
# we need to set up a few functions allowing us to measure the underlying differential equations

# Create a function to compute the derivative of the production function (see above equations)
def f_prime(x):
  return alpha * x**(alpha - 1)
# Create a function for the derivative of k(t) w.r.t time
def k_dot(k, c):
  return f(k) - c - (n + g) * k
# Create a function for the derivative of c(t) w.r.t. time
def c_dot(k, c):
  return (f_prime(k) - rho - theta * g) / theta * c

In [ ]:
# Set up the vector grid parameters
# Square of the number of points in our grid
n_points = 15
# Create a grid (for the points on the plot where vectors are drawn)
x, y = np.meshgrid(
    np.linspace(3, max(k), n_points), 
    np.linspace(0.001, max(c_star(k) * 1.5), n_points))
# Compute the x component of each vector
u = k_dot(x, y)
# Compute the y component of each vector
v = c_dot(x, y)
# Create a figure
fig, ax = plt.subplots(figsize = (14, 10))
# Add a green line with x values given by k and y values given by c*(k)
ax.plot(k, c_star(k), color = 'green', label = "k̇ = 0")
# Add a vertical line at k*
ax.axvline(k_star, color = 'blue', label = 'ċ = 0')
# Add a single black point at (x, y) = (k*, c*), this is our equilibrium
ax.plot([k_star], [c_star(k_star)], 'o', color = 'black')
# Annotate the point with the letter E
ax.annotate('E', xy = (k_star + 1, c_star(k_star)), size = 20)
# Add the vector field, showing how the dynamics of the system behave
ax.quiver(x, y, u, v, pivot = 'mid', color = 'gray', angles='xy', 
           scale_units='xy', scale = 0.25, minlength = 0.5)
ax.set_xlabel("k")                        # Add x-axis label
ax.set_ylabel("c")                        # Add y-axis label
ax.set_title("The dynamics of c and k")   # Add plot title
ax.grid()                                 # Add a grid in the background
ax.legend()                               # Add legend (show labels top right)

The above picture is very similar to figure 2.3 in Romer, however, it has the advantage of giving a better intuition about the full dynamics within the system. Indeed, it is much easier to see from this picture that some changes in the state of the economy will be much more drastic then others, depending on where the current state of the economy is.

We continue by building a function which, given an starting point $(k, c)$ on our plot, computes the evolution of the economy according to the law of motions defined in equation (2.24) and (2.25). The idea behind this function is a simple iteration through time, where at each time step we simply add the derivative $\dot{k}$ to the current $k$ and the derivative $\dot{c}$ to the current $c$.

In [ ]:
# Create a function which returns an array containing the trajectory of the 
# economy starting a (k, c) for T time periods
def trajectory(k, c, T = 1000):
  # Initialize 2 arrays for the evolution of k and c
  k = [k]
  c = [c] 
  # Iterate over the time periods and compute the evolution of k and c
  for t in range(T):
    # Compute the increments for k and c, saving them allows to reuse them
    # when checking the early stopping conditions
    kd = k_dot(k[t], c[t])
    cd = c_dot(k[t], c[t])
    # Append the increments to the existing lists of values
    k.append(k[t] + kd)
    c.append(c[t] + cd)
    # Early stopping condition
    if (kd == cd == 0) or (k[-1] <= 0) or (c[-1] <= 0): 
      break
  return np.asarray(k), np.asarray(c)

In [ ]:
# Finding a starting point which will end up in the equilibrium E is slightly more complicated
def reverse_trajectory(k_star, T = 150, eps = 0.01):
  # Initialize 2 arrays for the evolution of k and c but starting in [k_star, c_star]
  k = [k_star]
  c = [c_star(k_star)]
  # Add a slight perturbation (±eps%) from the equilibrium to get a point with non-zero derivatives
  k.append(k[0] * (1 + eps))
  c.append(c[0] * (1 + eps))
  for t in range(1, T):
    # Compute the increments for k and c towards the equilibrium and subtract them
    # from the current position
    kd = k_dot(k[t], c[t])
    cd = c_dot(k[t], c[t])
    k.append(k[t] - kd)
    c.append(c[t] - cd)
  # Return the reversed arrays for k and c
  return np.asarray(k[::-1]), np.asarray(c[::-1])

In [ ]:
# Iterate over some 'randomly' chosen starting points and plot the evolution of the economy
# Notice we do not create a new figure, we simply add the trajectories to the existing
# figure we previously created
for kc in [(5, 0.6, 'A'), (15, 0.6, 'B'), (50, 2.1, 'C'), (60, 2.0, 'D'), (175, 2.0, 'F')]:
  # Compute the x and y arrays using the trajectory function we defined above
  x, y = trajectory(kc[0], kc[1])
  ax.plot(x, y, '--', color = 'orange')         # Draw the paths as a dashed line
  ax.plot(kc[0], kc[1], 'o', color = 'orange')  # Add a dot at the starting point
  ax.annotate(kc[2], xy = kc[:2], size = 18)    # Add a letter at each starting point
# Add the trajectories which ends up in the equilibirum
x, y = reverse_trajectory(k_star, T = 150)
ax.plot(x, y, '--', color = 'purple')
ax.plot(x[0], y[0], 'o', color = 'purple')
ax.annotate('G', xy = (x[0], y[0]), size = 18)
x, y = reverse_trajectory(k_star, eps = -0.01, T = 160)
ax.plot(x, y, '--', color = 'purple')
ax.plot(x[0], y[0], 'o', color = 'purple')
ax.annotate('H', xy = (x[0], y[0]), size = 18)
# Restrain the y-axis so that it cuts off the paths of A and C as these 'explode'
ax.set_ylim(bottom = -0.1, top = max(c_star(k) * 1.5) + 0.1)
fig                                             # Display the figure

The above picture shows the direction for the path of the economy in different starting positions, but in a completely static manner.

We now add a time component using animations, i.e. we create an animation of how the economy develops over time starting in a given point.

In [ ]:
# Create a function to plot an animated evolution of the economy given a specific starting point
def animated_plot(k_start, c_start):
  # Create the plot
  fig = plt.figure(figsize = (12, 8))
  # Create a linear space of 1000 points between the two k values for which where c* = 0
  k = np.linspace(0, pow(n + g, 1 / (alpha - 1)), 1000)
  # Draw a green curve where k̇ = 0 and a blue line where ċ = 0
  plt.plot(k, c_star(k), color = 'green', label = "k̇ = 0")
  plt.axvline(k_star, color = 'blue', label = "ċ = 0")
  plt.xlabel("k")                       # Add x-axis label
  plt.ylabel("c")                       # Add y-axis label
  plt.title("The dynamics of c and k")  # Add plot title
  plt.grid()                            # Add a grid
  # Compute the evolution of k and c using our previously defined function
  k, c = trajectory(k_start, c_start, 1000)
  # Plot a dashed orange line (notice how it is initialized with empty arrays)
  evolution_line,  = plt.plot([], [], '--', label = 'Economy trajectory',
                              color = 'orange')
  # Define how the evolution_line (the orange dashed line) evolves as a function of time
  def animate(t):
    # Instead of taking steps with linearly increasing size, we use a bit of a 
    # trick to take smaller steps in the start and larger steps as time goes by
    step = int((t/50)**3 * len(k))
    # Update the evolution_line data
    evolution_line.set_data(k[:step], c[:step])
    return evolution_line,
  # Add legend and show the plot
  plt.legend()
  return animation.FuncAnimation(fig, animate, frames = 50, interval = 20, blit = True)

In [ ]:
# Create an animation of the economy's evolution starting at a point (k = 150, c = 1.4)
# Change the parameters of the function animated_plot() to see how different starting points behave
anim = animated_plot(150, 1.4)
plt.close()
anim